In [ ]:
import gzip
import shutil
import urllib.request
from tqdm import tqdm

In [ ]:
from utils.path_manage import get_files
data, lookup, ASD_dictionary, BCE_dictionary, _, __ = get_files()

In [ ]:
import torch
from torch_geometric.data import * 

In [ ]:
atom_set = {'C', 'Zn', 'S', 'Mg', 'Rb', 'Cs', 'La', 'Pb', 'Fe', '*', 'Br', 'Pd', 'Be', 'F', 'P', 'Sn', 'As', 'Eu', 'Mn', 'Cd', 'O', 'K', 'W', 'Ba', 'Sr', 'Tb', 'Cl', 'R', 'U', 'Cu', 'Li', 'Ca', 'A', 'Na', 'Si', 'Co', 'N', 'Al', 'H', 'Tl', 'B', 'I', 'D', 'Se', 'Ar', 'Ni', 'T'}

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
atoms = np.array(list(atom_set)).reshape(-1,1)
enc.fit(atoms)

In [ ]:
def get_graph_files(ASD_molecule):
    urllib.request.urlretrieve("http://202.120.143.190/ASD2015Common/static_file//modulator/mol_stru/mol_2d_gz/{}.mol.gz".format(ASD_molecule), 'Drug_zips/{}.mol.gz'.format(ASD_molecule)) 
    with gzip.open('Drug_zips/{}.mol.gz'.format(ASD_molecule), 'rb') as f_in:
        with open('Drug_files/{}.mol'.format(ASD_molecule), 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

def read_graph(ASD_molecule):
    with open('Drug_files/{}.mol'.format(ASD_molecule), 'r') as text:
        lines = text.readlines()
    return lines

def graph_info(lines, encoder = enc):
    no_atoms = int(lines[3].split()[0])
    no_bonds = int(lines[3].split()[1])
    atoms = [[atom.split()[3]] for atom in lines[4:no_atoms+4]]
    one_hot_atoms = enc.transform(np.array(atoms)).toarray()
    # print(one_hot_atoms)
    bonds = [[int(bond.split()[0]), int(bond.split()[1])] for bond in lines[no_atoms+4:no_atoms+no_bonds+4]]
    # print([bond.split() for bond in lines[no_atoms+4:no_atoms+no_bonds+4]])
    bond_weights = [float(bond.split()[2])/2 for bond in lines[no_atoms+4:no_atoms+no_bonds+4]]
    torch_atoms = torch.tensor(one_hot_atoms, dtype=torch.float)
    torch_bonds = torch.tensor(bonds, dtype=torch.long)
    torch_bond_weights = torch.tensor(bond_weights, dtype=torch.float)

    return Data(x=torch_atoms, edge_index=torch_bonds.t().contiguous(), edge_attr=torch_bond_weights) 

In [ ]:
ASD_molecule_list = list(set([lookup[x[0]] for x in data]))

In [ ]:
graph_list = []

for ASD_molecule in tqdm(ASD_molecule_list):
    try:
        with open('Drug_files/{}.mol'.format(ASD_molecule), 'r') as text:
            lines = text.readlines()
            graph_list.append(graph_info(lines))
    except:
        print('{} missing'.format(ASD_molecule))
        graph_list.append('{} missing'.format(ASD_molecule))

In [ ]:
# import pickle
# with open('data/drug_graph_list.pickle', 'wb') as handle:
#     pickle.dump(graph_list, handle)

In [ ]:
import pickle
with open('data/drug_graph_list.pickle', 'rb') as handle:
    graph_list_2 = pickle.load(handle)

In [ ]:
# no_atoms = int(lines[3].split()[0])
# no_bonds = int(lines[3].split()[1])
# bonds = [[int(bond.split()[0]), int(bond.split()[1])] for bond in lines[no_atoms+4:(no_atoms+no_bonds)+4]]
# bonds

In [ ]:
# re-download 37650 to 37758

# for ASD_molecule in tqdm(ASD_molecule_list[37650:37758]):
#     try:
#         get_graph_files(ASD_molecule)
#     except:
#         print(ASD_molecule, ' missing')

In [ ]:
# for ASD_molecule in tqdm(ASD_molecule_list[:10]):
#     # try:
#     with open('Drug_files/{}.mol'.format(ASD_molecule), 'r') as text:
#         lines = text.readlines()
#         graph = graph_info(lines)
#     # except:
    #     print('oof')

In [ ]:
# atom_set = set()
        # no_atoms = int(lines[3].split()[0])
        # atoms = [atom.split()[3] for atom in lines[4:no_atoms+4]]
        # [atom_set.add(atom) for atom in atoms]